In [44]:
import os
import pandas as pd

# 입력 파일 이름
#input_file_name = './data/chain-info.csv'
#input_file_name = './data/chain-market.csv'
#input_file_name = './data/chain-market-BTC.csv'
input_file_name = './result-files/market-chain.result.csv'

# Read the CSV file
df = pd.read_csv(input_file_name)

# ...
# invalid_rows = pd.to_datetime(df['_time'], errors='coerce').isna()
# print(df[invalid_rows])

def process_time(t):
    split_t = t.split('.')
    if len(split_t) > 1:
        if len(split_t[1]) == 1: 
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 2:
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 3:
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 4:
            return split_t[0] + '.000Z'
        elif len(split_t[1]) == 5:
            return split_t[0] + '.000Z'
    else:  # If there is no decimal point
        return t[:-1] + '.000Z'

df['_time'] = df['_time'].apply(process_time)

df['_time'] = pd.to_datetime(df['_time'])

# Round the '_time' column to the nearest hour
df['_time'] = df['_time'].dt.round('H')

# Convert back to the original timestamp format without the timezone offset
# df['_time'] = df['_time'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S.000Z'))

# Remove duplicate rows based on '_time' and 'chainName'
df = df.drop_duplicates(subset=['_time', 'chainName'])

# 파일의 기본 이름(디렉토리 없음)을 가져와서 파일 확장자 제거
base_file_name = os.path.basename(input_file_name)
base_file_name_without_extension = os.path.splitext(base_file_name)[0]

# 결과 파일이 저장될 디렉토리 정의
output_directory = './result-files/'

# 출력 파일 이름 생성
output_file_name = output_directory + base_file_name_without_extension + '.processed.csv'

# 누락된 값의 평균을 수정된 파일 이름의 텍스트 파일로 저장
df.to_csv(output_file_name, header=True)

In [26]:
import pandas as pd

# Step 1: Load each csv file into a separate DataFrame
chainInfo = pd.read_csv('./data/chain-info.csv')
chainMarket = pd.read_csv('./data/chain-market.csv')
chainMarketBTC = pd.read_csv('./data/chain-market-BTC.csv')

# Step 2: Select the required columns from each DataFrame
chainInfo = chainInfo[['apr', 'blockTime', 'bondedTokens', 'communityTokens', 'inflation', 'tps', '_time', 'chainName']]
# Add 'BTC' to each column name in chainMarketBTC
chainMarketBTC = chainMarketBTC[['cap', 'price', 'priceChangePercentage1H', 'totalVolume', '_time', 'chainName']]
chainMarketBTC.columns = [str(col) + 'BTC' if col not in ['_time', 'chainName'] else col for col in chainMarketBTC.columns]

# Step 3: Merge the DataFrames on '_time' and 'chainName' using inner merge
merged_df = pd.merge(chainMarket, chainInfo, on=['_time', 'chainName'], how='inner')
merged_df = pd.merge(merged_df, chainMarketBTC, on=['_time', 'chainName'], how='inner')

# Step 4: Save the result as a new csv file
merged_df.to_csv('./result-files/market-chain.result.csv', index=True)

In [33]:
mc_df = pd.read_csv('./result-files/market-chain.result.processed.csv')

cosmos = pd.read_csv('./result-files/ucosmos-balance.processed.plus.no5.csv')
kava = pd.read_csv('./result-files/ukava-balance.processed.plus.no5.csv')
juno = pd.read_csv('./result-files/ujuno-balance.processed.plus.no5.csv')
osmosis = pd.read_csv('./result-files/uosmo-balance.processed.plus.no5.csv')

#cosmos = pd.read_csv('./data/cosmos-balance.csv')
#kava = pd.read_csv('./data/kava-balance.csv')
#juno = pd.read_csv('./data/juno-balance.csv')
#osmosis = pd.read_csv('./data/osmosis-balance.csv')

# Step 1: Select the required columns from each DataFrame and add 'chainName'
cosmos = cosmos[['_time', 'moduleName', 'amountValue', 'boxplot']]
cosmos['chainName'] = 'cosmos'
kava = kava[['_time', 'moduleName', 'amountValue', 'boxplot']]
kava['chainName'] = 'kava'
juno = juno[['_time', 'moduleName', 'amountValue', 'boxplot']]
juno['chainName'] = 'juno'
osmosis = osmosis[['_time', 'moduleName', 'amountValue', 'boxplot']]
osmosis['chainName'] = 'osmosis'

# Reshape the dataframes
cosmos = cosmos.pivot_table(values=['amountValue', 'boxplot'], index=['_time', 'chainName'], columns='moduleName').reset_index()
kava = kava.pivot_table(values=['amountValue', 'boxplot'], index=['_time', 'chainName'], columns='moduleName').reset_index()
juno = juno.pivot_table(values=['amountValue', 'boxplot'], index=['_time', 'chainName'], columns='moduleName').reset_index()
osmosis = osmosis.pivot_table(values=['amountValue', 'boxplot'], index=['_time', 'chainName'], columns='moduleName').reset_index()

cosmos.columns = [f'{j}-{i}' if j else i for i,j in cosmos.columns]
kava.columns = [f'{j}-{i}' if j else i for i,j in kava.columns]
juno.columns = [f'{j}-{i}' if j else i for i,j in juno.columns]
osmosis.columns = [f'{j}-{i}' if j else i for i,j in osmosis.columns]

# Merge the dataframes
balance_df = pd.concat([cosmos, kava, juno, osmosis], ignore_index=True)
final_df = pd.merge(mc_df, balance_df, on=['_time', 'chainName'])
final_df = final_df.drop(['Unnamed: 0'], axis=1)
print(final_df.head())

# Step 4: Save the result as a new csv file
final_df.to_csv('./result-files/market-chain-balance.result.csv', index=True)

                       _time chainName    ath                   athDate   atl   
0  2023-01-24 21:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16  \
1  2023-01-24 22:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   
2  2023-01-24 23:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   
3  2023-01-25 00:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   
4  2023-01-25 01:00:00+00:00    cosmos  44.45  2022-01-17T00:34:41.497Z  1.16   

                    atlDate         cap  capChange24h  capChangeRatio24H   
0  2020-03-13T02:27:44.591Z  3826578089     -69338669           -1.77978  \
1  2020-03-13T02:27:44.591Z  3834398759     -44719269           -1.15282   
2  2020-03-13T02:27:44.591Z  3813167577     -31946650           -0.83084   
3  2020-03-13T02:27:44.591Z  3806684761     -14779955           -0.38676   
4  2020-03-13T02:27:44.591Z  3820293895       3705855            0.09710   

   capRank  ...  gov-boxplot  not_bonded_tokens_pool-box